yfinance API Documentation: https://ranaroussi.github.io/yfinance/reference/index.html 

git repo: https://github.com/tomwmoore/tm-finance-lab

In [22]:
import sys 
import os

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

import importlib


# Add root path so other subfolders are accessible
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


import src.utils.db_azure
# Reload it every time the cell is run
importlib.reload(src.utils.db_azure)

from src.utils.db_azure import get_analytics_azure_engine
from src.utils.db_azure import get_dashboard_azure_engine
from src.utils.db_azure import azure_upsert


import src.utils.plots
# Reload it every time the cell is run
importlib.reload(src.utils.plots)

from src.utils.plots import plot_stock

import src.utils.indicators
# Reload it every time the cell is run
importlib.reload(src.utils.indicators)

from src.utils.indicators import compute_rsi


In [ ]:
    # Get list of tickers from given industry from asset header
query_symbol = f"""
                select symbol 
                from asset_header
                where industry = 'Oil & Gas E&P'
                """

engine = get_analytics_azure_engine()

symbols = pd.read_sql(query_symbol,engine)
symbols['symbol'].tolist()

In [11]:
# get some stock data
tickers = symbols['symbol'].tolist()
# data = yf.download(ticker, start="2025-01-01", end="2025-09-25")
data = yf.download(tickers, interval= '1d', group_by='ticker', start = '2024-01-01', end = '2025-01-01' )

# Clean up multi-index column names to just be the feature names (close, open etc) & make lowercase
data = data.stack(level=0,future_stack=True).rename_axis(['Date', 'Ticker']).reset_index(level=1)

# remote date as the index
data = data.reset_index()

data.columns = [col.lower() for col in data.columns]

# add current timestamp for future QC
data['updated_at'] = pd.Timestamp.now()




C:\Users\tomwm\AppData\Local\Temp\ipykernel_24376\3556350604.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, interval= '1d', group_by='ticker', start = '2024-01-01', end = '2025-01-01' )
[*********************100%***********************]  50 of 50 completed


In [11]:
from sqlalchemy.dialects.mssql import DATETIME2, VARCHAR, FLOAT, BIGINT

# Create new table and load to sql
azure_engine = get_analytics_azure_engine()


dtype_mapping = {
    'date': DATETIME2(6),        # precise for milliseconds
    'ticker': VARCHAR(20),       
    'open': FLOAT,
    'high': FLOAT,
    'low': FLOAT,
    'close': FLOAT,
    'volume': BIGINT,            
    'updated_at': DATETIME2(0)   # drops seconds, rounds to nearest minute
}

data.to_sql(
    'stock_prices',
    azure_engine,
    if_exists='fail', 
    index=False,
    dtype=dtype_mapping
)

165

In [14]:
# Create indices on new table
from sqlalchemy import text

with azure_engine.begin() as conn:
    conn.execute(text('alter table stock_prices alter column date datetime2 NOT NULL'))
    conn.execute(text('alter table stock_prices alter column ticker varchar(20) NOT NULL'))
    conn.execute(text('alter table stock_prices add constraint pkey_stock_prices PRIMARY KEY (date,ticker)'))

In [24]:
# upsert dataframe to postgres
azure_engine = get_analytics_azure_engine()
azure_upsert(data,azure_engine,'stock_prices')

In [2]:
# get data from postgres db
symbol = 'TOU.TO'
azure_engine = get_analytics_azure_engine()
select_query = f"select * from stock_prices where symbol = '{symbol}'"

df = pd.read_sql(select_query,azure_engine)

df.head()

,date,symbol,open,high,low,close,volume,updated_at
0,2020-01-02,TOU.TO,10.438542,10.519882,10.086072,10.221638,917200,2025-10-08 22:14:16
1,2020-01-03,TOU.TO,10.411428,10.580885,10.235193,10.418206,1252000,2025-10-08 22:14:16
2,2020-01-06,TOU.TO,10.472431,10.784231,10.465652,10.723227,2403400,2025-10-08 22:14:16
3,2020-01-07,TOU.TO,10.723228,10.743564,10.438541,10.567328,967700,2025-10-08 22:14:16
4,2020-01-08,TOU.TO,10.675784,10.906246,10.540219,10.743567,2926400,2025-10-08 22:14:16


In [ ]:
# Get rolling average
df['rolling_avg'] = df['close'].rolling(window=15).mean()

df_plot = df.copy()

df_plot = df[df['date'] > '2025-01-01']

# visualize prices
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = df_plot['date'], y = df_plot['close'],
    name = 'price',
    yaxis = 'y1',
    mode = 'lines+markers',
    line = dict(color='black')

))

fig.add_trace(go.Scatter(
    x = df_plot['date'], y = df_plot['rolling_avg'],
    name = 'rolling_avg',
    yaxis = 'y1',
    mode = 'lines',
    line = dict(color='red')

))



# Layout with multiple y-axes
fig.update_layout(
    title=f'Stock: {symbol}',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Prices', side='left'),
    legend=dict(x=0.01, y=0.99)
)

fig.show()


In [19]:
# calc indicators

df_plot = df.copy()

df_plot = df_plot[df_plot['date'] > '2025-01-01']

df_plot['rolling_avg'] = df_plot['close'].rolling(window=15).mean()
df_plot['upper_band'] = df_plot['rolling_avg'] + 2*df_plot['close'].rolling(window=15).std()
df_plot['lower_band'] = df_plot['rolling_avg'] - 2*df_plot['close'].rolling(window=15).std()



fig = plot_stock(df_plot, 
                 date_col='date', 
                 price_col='close',
                 indicators= {
                    'Rolling Avg (15d)': {'data': df_plot['rolling_avg'], 'color': 'red'},
                    'Upper Band': {'data': df_plot['upper_band'], 'color': 'green', 'dash': 'dot'},
                    'Lower Band': {'data': df_plot['lower_band'], 'color': 'green', 'dash': 'dot'}
                })
fig.show()

In [23]:
compute_rsi(df_plot['close'],period=15)

1255          NaN
1256          NaN
1257          NaN
1258          NaN
1259          NaN
          ...    
1446    58.522727
1447    51.333331
1448    50.293377
1449    47.058825
1450    45.731700
Name: close, Length: 196, dtype: float64